In [1]:
import pandas as pd

### Trade Data

Filter solely on the year 2005 and filter out irrelevant columns. Also express the import/export in more relative terms. 
If flow values are missing (-9), they are imputed with 0. If a row has zero flow in both directions, it is removed from the data.

In [2]:
year = 2005
cols = ['ccode1', 'ccode2', 'flow1', 'flow2']

In [3]:
def calc_relative_import(x, row, n):
    """
    Calculates relative import of a specific flow for a country {n} (1/2).
    """ 
    return row[f'flow{n}'] / (x[x['ccode1'] == row[f'ccode{n}']]['flow1'].sum() + x[x['ccode2'] == row[f'ccode{n}']]['flow2'].sum())


def calc_relative_export(x, row, n):
    """
    Calculates relative export of a specific flow for a country {n} (1/2).
    """
    e = 2 if n == 1 else 1
    return row[f'flow{e}'] / (x[x['ccode1'] == row[f'ccode{n}']]['flow2'].sum() + x[x['ccode2'] == row[f'ccode{n}']]['flow1'].sum())

#### Undirected

In [4]:
trade_df = (
    pd.read_csv('trade_dyadic.csv')
    .pipe(lambda x: x[x['year'] == year])
    .pipe(lambda x: x[cols].replace({-9: 0}))
    .pipe(lambda x: x[~((x['flow1'] == 0) & (x['flow2'] == 0))])
    .pipe(lambda x: x.assign(
        # value of import of country1 from country2 relative to total import of country1
        relative_import1 = x.apply(lambda row: calc_relative_import(x, row, 1), axis=1),
        # value of import of country2 from country1 relative to total import of country2
        relative_import2 = x.apply(lambda row: calc_relative_import(x, row, 2), axis=1),
        # value of export of country1 to country2 relative to total export of country1
        relative_export1 = x.apply(lambda row: calc_relative_export(x, row, 1), axis=1),
        # value of export of country2 to country1 relative to total export of country 2
        relative_export2 = x.apply(lambda row: calc_relative_export(x, row, 2), axis=1),       
    ))
    .reset_index(drop=True)
)
# calculate metric of trade_importance
trade_df['trade_importance'] = (trade_df['relative_import1'] + trade_df['relative_import2'] + trade_df['relative_export1'] + trade_df['relative_export2']) / 4

print(f"Shape of trade_df: {trade_df.shape}\n")
trade_df.to_csv('results/trade_relations_undir.csv', index=False)
trade_df.head()

Shape of trade_df: (11934, 9)



,ccode1,ccode2,flow1,flow2,relative_import1,relative_import2,relative_export1,relative_export2,trade_importance
0,2,20,291943.50000,195360.28000,0.168938,0.570128,0.204894,0.802721,0.436670
1,2,31,726.29999,1945.79000,0.000420,0.218027,0.002041,0.297894,0.129596
2,2,40,0.00000,397.87000,0.000000,0.056731,0.000417,0.000000,0.014287
3,2,41,458.50000,756.90997,0.000265,0.497009,0.000794,0.815954,0.328505
4,2,42,4721.39990,5005.60990,0.002732,0.469146,0.005250,0.770421,0.311887


#### Directed

In [5]:
dir_trade_df = (
    trade_df.append(
        trade_df
        .pipe(lambda x: x.rename(columns={'ccode1': 'ccode2', 'ccode2': 'ccode1', 
                                          'flow1': 'flow2', 'flow2': 'flow1', 
                                          'relative_import1': 'relative_import2', 'relative_import2': 'relative_import1',
                                          'relative_export1': 'relative_export2', 'relative_export2': 'relative_export1'})))
    .pipe(lambda x: x.rename(columns={'flow1': 'import_flow1'}))
    .drop(columns=['flow2', 'relative_import2', 'relative_export1'])
    .reset_index(drop=True)
)
print(f"Shape of dir_trade_df: {dir_trade_df.shape}\n")
dir_trade_df.to_csv('results/trade_relations_dir.csv', index=False)
dir_trade_df.head()

Shape of dir_trade_df: (23868, 6)



,ccode1,ccode2,import_flow1,relative_import1,relative_export2,trade_importance
0,2,20,291943.50000,0.168938,0.802721,0.436670
1,2,31,726.29999,0.000420,0.297894,0.129596
2,2,40,0.00000,0.000000,0.000000,0.014287
3,2,41,458.50000,0.000265,0.815954,0.328505
4,2,42,4721.39990,0.002732,0.770421,0.311887


### Country Codes

Remove the duplicates, and the countries that are not in the trade data (i.e. countries that no longer exist). 

In [6]:
country_codes_df = (
    pd.read_csv('country_codes_gdp.csv', sep=';')
    .drop_duplicates()
    .pipe(lambda x: x.rename(str.lower, axis='columns'))
    .pipe(lambda x: x.rename(columns={'stateabb': 'cabb', 'statenme': 'cname'}))
    .pipe(lambda x: x[x['ccode'].isin(set(dir_trade_df['ccode1'].unique().tolist()))])
    .reset_index(drop=True)
)
print(f"Shape of country_codes_df: {country_codes_df.shape}\n")
country_codes_df.to_csv('results/country_data.csv', index=False)
country_codes_df.head(10)

Shape of country_codes_df: (183, 4)



,cabb,ccode,cname,gdp2005
0,USA,2,United States of America,1.175000e+13
1,CAN,20,Canada,1.023000e+12
2,BHM,31,Bahamas,5.295000e+09
3,CUB,40,Cuba,3.392000e+10
4,HAI,41,Haiti,1.205000e+10
5,DOM,42,Dominican Republic,5.568000e+10
6,JAM,51,Jamaica,1.113000e+10
7,TRI,52,Trinidad and Tobago,1.148000e+10
8,BAR,53,Barbados,4.569000e+09
9,DMA,54,Dominica,3.840000e+08


### Diplomatic Relations Data

Filter on year 2005, rename columns and remove countries that are not in the other two datasets.

#### Undirected

In [7]:
diplomatic_df = (
    pd.read_csv('diplomatic_exchange.csv')
    .pipe(lambda x: x[x['year'] == year])
    .pipe(lambda x: x.rename(str.lower, axis='columns'))
    .pipe(lambda x: x[((x['ccode1'].isin(set(dir_trade_df['ccode1'].unique().tolist()))) & 
                       (x['ccode2'].isin(set(dir_trade_df['ccode1'].unique().tolist()))))])
    .drop(columns=['version', 'year'])
    .reset_index(drop=True)
)
print(f"Shape of diplomatic_df: {diplomatic_df.shape}\n")
diplomatic_df.to_csv('results/diplomatic_relations_undir.csv', index=False)
diplomatic_df.head()

Shape of diplomatic_df: (33306, 5)



,ccode1,ccode2,dr_at_1,dr_at_2,de
0,2,20,3,3,1
1,2,31,3,3,1
2,2,40,9,9,0
3,2,41,3,3,1
4,2,42,3,3,1


In [8]:
new_diplomatic_df = diplomatic_df[['ccode1', 'ccode2', 'de']]

print(f"Shape of diplomatic_df: {new_diplomatic_df.shape}\n")
new_diplomatic_df.to_csv('results/new_diplomatic_relations_undir.csv', index=False)
new_diplomatic_df.head()

Shape of diplomatic_df: (33306, 3)



,ccode1,ccode2,de
0,2,20,1
1,2,31,1
2,2,40,0
3,2,41,1
4,2,42,1


#### Directed

In [9]:
dir_diplomatic_df = (
    diplomatic_df.append(
        diplomatic_df
        .pipe(lambda x: x.rename(columns={'ccode1': 'ccode2', 'ccode2': 'ccode1', 
                                          'dr_at_1': 'dr_at_2', 'dr_at_2': 'dr_at_1'})))
    .drop(columns=['de', 'dr_at_2'])
    .reset_index(drop=True)
)
print(f"Shape of dir_diplomatic_df: {dir_diplomatic_df.shape}\n")
dir_diplomatic_df.to_csv('results/diplomatic_relations_dir.csv', index=False)
dir_diplomatic_df.head()

Shape of dir_diplomatic_df: (66612, 3)



,ccode1,ccode2,dr_at_1
0,2,20,3
1,2,31,3
2,2,40,9
3,2,41,3
4,2,42,3
